In [1]:
import pandas as pd

employees_old = r"C:\Users\Narasimha\Desktop\Agent2_sample\scrapers\data\raw\employees_past_week.csv"
employees_new = r"C:\Users\Narasimha\Desktop\Agent2_sample\scrapers\data\raw\employees_present_week.csv"


In [2]:
# -------------------------
# CONFIG
# -------------------------
USECOLS = [0, 1, 2, 3, 4]  # only first 5 columns
DTYPES = {
    0: "string",
    1: "string",
    2: "string",
    3: "string",
    4: "string"
}

In [3]:
# -------------------------
# LOAD CSVs (FAST)
# -------------------------
old_df = pd.read_csv(
    employees_old,
    usecols=USECOLS,
    dtype=DTYPES,
    engine="c",
)

new_df = pd.read_csv(
    employees_new,
    usecols=USECOLS,
    dtype=DTYPES,
    engine="c",
)

old_df.columns = ["email_id", "name", "role", "company_name", "company_location"]
new_df.columns = ["email_id", "name", "role", "company_name", "company_location"]


In [4]:
# -------------------------
# CLEAN DATA (VECTORISED)
# -------------------------
for df in (old_df, new_df):
    df["email_id"] = df["email_id"].str.strip().str.lower()
    df["role"] = df["role"].str.strip()
    df["company_name"] = df["company_name"].str.strip()
    df["company_location"] = df["company_location"].str.strip().fillna("")


In [5]:
# -------------------------
# MERGE ONLY NEEDED COLUMNS
# -------------------------
merged = old_df[
    ["email_id", "role", "company_name", "company_location"]
].merge(
    new_df[["email_id", "role", "company_name"]],
    on="email_id",
    how="inner",
    suffixes=("_old", "_new")
)

In [8]:
# -------------------------
# FILTER CHANGED ROWS (VECTORIZED)
# -------------------------
company_changed = merged["company_name_old"] != merged["company_name_new"]
role_changed = merged["role_old"] != merged["role_new"]

changed_rows = merged[company_changed | role_changed].copy()


In [9]:
# -------------------------
# STATUS (NO APPLY)
# -------------------------
changed_rows["Status"] = "role changed"
changed_rows.loc[company_changed, "Status"] = "company changed"


In [10]:
# -------------------------
# FINAL REPORT
# -------------------------
changed_rows["Company (Location)"] = (
    changed_rows["company_name_old"]
    + " ("
    + changed_rows["company_location"].replace("", "Unknown")
    + ")"
)

new_dataset = changed_rows[
    ["email_id", "Company (Location)", "role_old", "Status"]
].rename(
    columns={
        "email_id": "Email",
        "role_old": "Position"
    }
)

new_dataset = new_dataset.reset_index(drop=True)
new_dataset.index += 1

In [11]:
print(new_dataset)


                     Email             Company (Location)  \
1         chris.p@apex.com        Apex Solutions (Dublin)   
2     sophia.t@everest.com    Everest Consulting (Zurich)   
3    victoria.n@lumina.com      Lumina Designs (New York)   
4        mark.d@fusion.com        Fusion Robotics (Tokyo)   
5      paula.s@quantum.com         Quantum AI (Singapore)   
6       brian.c@altair.com      Altair Security (Chicago)   
7      rachel.p@cypher.com             Cypher AI (London)   
8     hailey.d@monarch.com    Monarch Industries (London)   
9        colin.b@astro.com          Astro Cloud (Chicago)   
10      ariana.l@polar.com        Polar Studios (Toronto)   
11   sienna.b@crescent.com  Crescent Solutions (New York)   
12    logan.k@serenity.com       Serenity Health (London)   
13       laura.d@avant.com              Avant BI (London)   
14    julian.h@maxwell.com  Maxwell Investments (Toronto)   
15  tristan.h@datacore.com              DataCore (Boston)   

                       